In [1]:
# Tensorflow 2.3でメモリを指定及び節約して使うためのおまじない。
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[4], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[4], True)

import numpy as np
import random

np.random.seed(219)
random.seed(219)
tf.random.set_seed(219)

In [2]:
import os
dirpath = "/workdir/taki_lab/tiny-imagenet-200/"
train_dir = os.path.join(dirpath, "train")
val_dir = os.path.join(dirpath, "val")
test_dir = os.path.join(dirpath, "test")

In [3]:
from keras.preprocessing.image import ImageDataGenerator

N_TRAIN = 100000
N_VAL = 10000
INPUT_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(rescale=1.0/255,
                              rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest")

val_gen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_gen.flow_from_directory(train_dir,
                                               target_size=INPUT_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

val_generator = val_gen.flow_from_directory(val_dir,
                                           target_size=INPUT_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode="categorical")

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [4]:
from keras import layers, Model

def bottleneck(x, in_ch, out_ch, strides=1):
    params = {
        "kernel_initializer": "he_normal",
        "use_bias": False
    }
    
    inter_ch = out_ch // 4
    h1 = layers.BatchNormalization()(x)
    h1 = layers.ReLU()(h1)
    h1 = layers.Conv2D(inter_ch, kernel_size=1, strides=strides, padding="valid", **params)(h1)
    h1 = layers.BatchNormalization()(h1)
    h1 = layers.ReLU()(h1)
    h1 = layers.Conv2D(inter_ch, kernel_size=3, strides=1, padding="same", **params)(h1)
    h1 = layers.BatchNormalization()(h1)
    h1 = layers.ReLU()(h1)
    h1 = layers.Conv2D(out_ch, kernel_size=1, strides=1, padding="valid", **params)(h1)
    
    if in_ch != out_ch:
        h2 = layers.BatchNormalization()(x)
        h2 = layers.Conv2D(out_ch, kernel_size=1, strides=strides, padding="same", **params)(h2)
    else:
        h2 = x
    
    h = layers.Add()([h1, h2])

    return h

def functional_resnet50(input_shape, output_size):
    inputs = layers.Input(input_shape)
    x = layers.BatchNormalization()(inputs)
    x = layers.ReLU()(x)
    x = layers.Conv2D(64, 7, 2, padding="same", kernel_initializer="he_normal", use_bias=False)(x)
    x = layers.MaxPool2D(3, 2, padding="same")(x)
    
    x = bottleneck(x, 64, 256)
    x = bottleneck(x, 256, 256)
    x = bottleneck(x, 256, 256)
    
    x = layers.Conv2D(512, kernel_size=1, strides=2)(x)
    x = bottleneck(x, 512, 512)
    x = bottleneck(x, 512, 512)
    x = bottleneck(x, 512, 512)
    x = bottleneck(x, 512, 512)
    
    x = layers.Conv2D(1024, kernel_size=1, strides=2, use_bias=False)(x)
    x = bottleneck(x, 1024, 1024)
    x = bottleneck(x, 1024, 1024)
    x = bottleneck(x, 1024, 1024)
    x = bottleneck(x, 1024, 1024)
    x = bottleneck(x, 1024, 1024)
    x = bottleneck(x, 1024, 1024)
    
    x = layers.Conv2D(2048, kernel_size=1, strides=2, use_bias=False)(x)
    x = bottleneck(x,2048, 2048)
    x = bottleneck(x,2048, 2048)
    x = bottleneck(x,2048, 2048)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation="relu")(x)
    outputs = layers.Dense(output_size, activation="softmax", kernel_initializer="he_normal")(x)
    return Model(inputs=inputs, outputs=outputs)

In [ ]:
from keras import optimizers
import wandb
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# wandbの設定
wandb.init(project="tiny-image-net", name="resnet50_pre-act_lr")
config = wandb.config
config.learning_rate = 0.0001
config.lr_factor = 0.5
config.optimaizer = "Adam"
config.batch_size = BATCH_SIZE
config.input_size = INPUT_SIZE
config.rotation_range = 10
config.width_shift_range = 0.1
config.height_shift_range = 0.1
config.shear_range = 0.1
config.zoom_range = 0.1

# 最適化設定
model = functional_resnet50((224, 224, 3), 200)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
             loss="categorical_crossentropy",
             metrics=["acc"])

early_stopping = EarlyStopping(monitor="val_acc", patience=5)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)

callback_list = [early_stopping, reduce_lr, WandbCallback()]


# 学習
history = model.fit(train_generator,
    steps_per_epoch=N_TRAIN//BATCH_SIZE,
    epochs=1000,
    validation_data=val_generator,
    validation_steps=N_VAL//BATCH_SIZE,
    use_multiprocessing=True,
    workers=24,
    callbacks=callback_list)

wandb: Wandb version 0.10.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/1000
3125/3125 [==============================] - 644s 206ms/step - loss: 4.8145 - acc: 0.0521 - val_loss: 5.6470 - val_acc: 0.0417
Epoch 2/1000
3125/3125 [==============================] - 640s 205ms/step - loss: 4.1227 - acc: 0.1228 - val_loss: 4.1008 - val_acc: 0.1311
Epoch 3/1000
3125/3125 [==============================] - 648s 207ms/step - loss: 3.7161 - acc: 0.1810 - val_loss: 3.7034 - val_acc: 0.1920
Epoch 4/1000
3125/3125 [==============================] - 641s 205ms/step - loss: 3.4142 - acc: 0.2288 - val_loss: 3.4510 - val_acc: 0.2318
Epoch 5/1000
3125/3125 [==============================] - 637s 204ms/step - loss: 3.1555 - acc: 0.2740 - val_loss: 3.4949 - val_acc: 0.2445
Epoch 6/1000
3125/3125 [==============================] - 639s 204ms/step - loss: 2.9380 - acc: 0.3125 - val_loss: 3.0996 - val_acc: 0.2925
Epoch 7/1000
3125/3125 [==============================] - 640s 205ms/step - loss: 2.7426 - acc: 0.3521 - val_loss: 2.9259 - val_acc: 0.3290
Epoch 8/1000
3125/31